In [122]:
import numpy as np
import pandas as pd
import math
import time

#data processing

no_bit = 5 # no of nodes in input layer( no of features )

# make data with n bits
import itertools
ds = []
word = '0'*no_bit
ds.append(word)
[
    ds.append(num)
    for num in (''.join(p) for p in itertools.product('01', repeat=no_bit)) 
    if not num.startswith(word)
]

#print("The dataset is created with",no_bit,"bits are of length",len(ds))
#print(ds)

# insert desired output for each sample
data = []

for string in ds:
    l = []
    for char in string:
        l.append(int(char))
    if(l[0]==0):
        l.append(0)
    else:
        l.append(1)
    data.append(l)
    
# add column names for each feature
column = []
for i in range(len(data[0])-1):
    column.append("X"+str(i+1))
column.append("Output")
#column
#data
df = pd.DataFrame(data, columns=column)
#df
df['Output'].value_counts()

hidden_layer = 1 
nodes_in_hidden_layer = 2
class_no = len(set(df['Output']))

In [123]:
k1,k2,eta1,eta2 =1,1,1,1     

nodes_in_output_layer =  int(math.log2(class_no))

if((math.ceil(math.log2(class_no)) != math.floor(math.log2(class_no)))):
    nodes_in_output_layer+=1

#print(nodes_in_output_layer)

def initializeWeightThresh():
    Wij = [list(np.random.rand(nodes_in_hidden_layer)) for i in range(len(data[0])-1)]
    Uhj = list(np.random.rand(nodes_in_hidden_layer))
    Wjk = [list(np.random.rand(nodes_in_output_layer)) for i in range(nodes_in_hidden_layer)]
    Uok = list(np.random.rand(nodes_in_output_layer))
    return Wij,Uhj,Wjk,Uok

Wij = [ list(np.random.rand(nodes_in_hidden_layer)) for i in range(len(data[0])-1)]            #input to hidden weights Wij
Uhj = list(np.random.rand(nodes_in_hidden_layer))                                              # threshold of hidden layer's nodes
Wjk = [ list(np.random.rand(nodes_in_output_layer)) for i in range(nodes_in_hidden_layer)]    #weights of hidden to output Wjk
Uok = list(np.random.rand(nodes_in_output_layer))                                             # threshold of output layer's nodes
Oi,Oj,Ok,sigmak = 0,0,0,0

# sigmoid function 1/1+e^(-k*active)
def sigmoid(k,activ):
    return 1/(1+math.exp(-k*activ))

In [124]:
def computeOutputHidden(row):
    netj = [] 
    Oj = []   
    activj = [] 
    for j in range(nodes_in_hidden_layer): 
        net = 0.0
        for i in range(len(row)-1):
            net += Wij[i][j] * row[i]    # equation-1 
        active = net + Uhj[j]            # equation-2 
        output = sigmoid(k1,active)      # equation-3 
        netj.append(net) 
        activj.append(active)
        Oj.append(output)
    return Oj 

Oj = computeOutputHidden([0,0,1,0])
#Oj

def computeOutputOutput(Oj):
    netk = [] 
    Ok = [] 
    activk = []
    for k in range(nodes_in_output_layer): 
        net = 0.0
        for j in range(nodes_in_hidden_layer):
            net += Wjk[j][k] * Oj[j]   # equation-4
        active = net + Uok[k]          # equation-5 
        output = sigmoid(k2,active)    # equation-6 
        netk.append(net) 
        activk.append(active)
        Ok.append(output)
    return Ok  
Ok = computeOutputOutput([0,1,1,1])
#Ok

def computeSigma(Ok,tak):                                  #sigmak = tk - ok
    sigmak = []
    for k in range(nodes_in_output_layer):   
        sigma = (tak - Ok[k])
        sigmak.append(sigma)
    return sigmak 

def adjustWeightsHiddenOutput():
    weight_diff = [eta2*k2*sigmak[k]*Oj[j]*Ok[k]*(1-Ok[k]) for j in range(nodes_in_hidden_layer) for k in range(nodes_in_output_layer)]    #equation-8 & 9
    for j in range(nodes_in_hidden_layer):
        for k in range(nodes_in_output_layer):
            Wjk[j][k]+=weight_diff[k]
            
def adjustThreshHiddenOutput():
    thresh_diff = [0]*nodes_in_output_layer
    for k in range(nodes_in_output_layer):
        thresh_diff[k] = eta2*k2*sigmak[k]*Ok[k]*(1-Ok[k])  #equation-10
        Uok[k]+=thresh_diff[k]  # equation-11  

def adjustWeightsInputHidden(row):
    Oi = row[:-1] 
    sops = [] 
    for j in range(nodes_in_hidden_layer):
        sop = 0.0
        for k in range(nodes_in_output_layer):
            sop = sop + (sigmak[k]*Wjk[j][k])
        sops.append(sop)
    for i in range(len(Oi)):
        for j in range(len(Oj)):
            weight_diff = eta1*k1*Oi[i]*Oj[j]*(1-Oj[j])*sops[j]       #equation-12
            Wij[i][j] += weight_diff                                  #equation-13
            
def adjustThreshInputHidden():
    sops = [] 
    for j in range(nodes_in_hidden_layer):
        sop = 0.0
        for k in range(nodes_in_output_layer):
            sop = sop + (sigmak[k]*Wjk[j][k])
        sops.append(sop)
    for j in range(len(Oj)):
        thresh_diff = eta1*k1*Oj[j]*(1-Oj[j])*sops[j]       #equation-14 
        Uhj[j] += thresh_diff                               #equation-15

In [125]:
#data

def split_predict_compare(train_percent,test_percent):
    no_of_train_samples = int(len(data)*train_percent*0.01)
    no_of_test_samples = int(len(data)*test_percent*0.01)
    train = data[:no_of_train_samples]
    test = data[-no_of_test_samples:]
    return train,test

In [129]:
train,test = split_predict_compare(80,20)
Wij,Uhj,Wjk,Uok = initializeWeightThresh() # intialize all weights and thresholds
#print("no of training samples are, ", len(train))
start_time = time.time() # note starting time 

i=0
while(i<len(train)):
    row = train[i] 
    Oj = computeOutputHidden(row) 
    Ok = computeOutputOutput(Oj) 
    
    sigmak = computeSigma(Ok,row[-1]) 
    adjustWeightsHiddenOutput() 
    adjustThreshHiddenOutput() 
    adjustWeightsInputHidden(row) 
    adjustThreshInputHidden() 
    if(row[-1]!=np.round(Ok[0])): 
        i=0
        continue
    else:
        i=i+1
end_time = time.time()
#print("training time, ",end_time-start_time," seconds")
cnt=0
for row in test: 
    Oj = computeOutputHidden(row) 
    Ok = computeOutputOutput(Oj) 
    if(row[-1]==np.round(Ok[0])): 
        cnt+=1
print("Accuracy ",cnt/len(test)*100.0)

Accuracy  83.33333333333334


In [127]:
train,test = split_predict_compare(70,30)
Wij,Uhj,Wjk,Uok = initializeWeightThresh() # intialize all weights and thresholds
#print("no of training samples are, ", len(train))
start_time = time.time() # note starting time 

i=0
while(i<len(train)):
    row = train[i] 
    Oj = computeOutputHidden(row) 
    Ok = computeOutputOutput(Oj) 
    
    sigmak = computeSigma(Ok,row[-1]) 
    adjustWeightsHiddenOutput() 
    adjustThreshHiddenOutput() 
    adjustWeightsInputHidden(row) 
    adjustThreshInputHidden() 
    if(row[-1]!=np.round(Ok[0])): 
        i=0
        continue
    else:
        i=i+1
end_time = time.time()
#print("training time, ",end_time-start_time," seconds")
cnt=0
for row in test: 
    Oj = computeOutputHidden(row) 
    Ok = computeOutputOutput(Oj) 
    if(row[-1]==np.round(Ok[0])): 
        cnt+=1
print("Accuracy ",cnt/len(test)*100.0)

Accuracy  66.66666666666666


In [132]:
train,test = split_predict_compare(65,35)
Wij,Uhj,Wjk,Uok = initializeWeightThresh() # intialize all weights and thresholds
#print("no of training samples are, ", len(train))
start_time = time.time() # note starting time 

i=0
while(i<len(train)):
    row = train[i] 
    Oj = computeOutputHidden(row) 
    Ok = computeOutputOutput(Oj) 
    
    sigmak = computeSigma(Ok,row[-1]) 
    adjustWeightsHiddenOutput() 
    adjustThreshHiddenOutput() 
    adjustWeightsInputHidden(row) 
    adjustThreshInputHidden() 
    if(row[-1]!=np.round(Ok[0])): 
        i=0
        continue
    else:
        i=i+1
end_time = time.time()
#print("training time, ",end_time-start_time," seconds")
cnt=0
for row in test: 
    Oj = computeOutputHidden(row) 
    Ok = computeOutputOutput(Oj) 
    if(row[-1]==np.round(Ok[0])): 
        cnt+=1
print("Accuracy ",cnt/len(test)*100.0)

Accuracy  45.45454545454545
